In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.cm import rainbow
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit 

## Step 1: Download the Heart Failure Prediction Dataset


In [ ]:
# Download the dataset (returns the path to the dataset folder)
path = kagglehub.dataset_download("fedesoriano/heart-failure-prediction")

# Load the dataset with pandas
dataset = pd.read_csv(dataset_path)

dataset


## Step 2: Dataset Preparation

In [ ]:
#Setting a random seed ensures that any random processes produce the same results each time the code is run.
import random
random.seed(42)

### Feature Preprocessing

In [ ]:
X = dataset.drop(columns="HeartDisease")  # All columns except the target
y = dataset["HeartDisease"]               # Target column

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()   # re
# Normalize only numeric columns
for col in X.columns:
    if pd.api.types.is_numeric_dtype(X[col]):
        X[col] = scaler.fit_transform(X[[col]])



### Train/Validation/Test Split

In [ ]:
X_train, X_test_val, y_train, y_test_val = train_test_split(
    X, y, 
    random_state=42,   # For reproducibility ------> no need for shuffling
    test_size=0.3,     # 30% for testing and validation, 70% for training
     stratify=y
)

X_test, X_val, y_test, y_val = train_test_split(
    X_test_val, y_test_val, 
    random_state=42,   # For reproducibility ------> no need for shuffling
    test_size=1/3,     # 20% for testing, 10% for validation of the whole dataset
     stratify=y_test_val
)

#ratio between number of zeros and ones are the same for all sets
print(y_test.value_counts())
print(y_val.value_counts())
print(y_train.value_counts())